In [58]:
import sys
import warnings
import re

warnings.simplefilter(action="ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

DATA_PATH = "../data"

In [59]:
# Load excel file
df = pd.read_excel(DATA_PATH + "/consumo_material_clean.xlsx")

## Preprocessing

In [60]:
# Separate code into two columns
new_columns = df["CODIGO"].str.extract(r"([a-zA-Z]+)([0-9]+)", expand=False)
df["CODIGO_CLASS"] = new_columns[0]
df["CODIGO_NUM"] = new_columns[1]
df.drop(columns=["CODIGO"], inplace=True)

In [61]:
# FECHAPEDIDO to datetime in day/month/year format
df["FECHAPEDIDO"] = pd.to_datetime(df["FECHAPEDIDO"], dayfirst=True)
df.sort_values(by=["FECHAPEDIDO"], inplace=True)
df.reset_index(drop=True, inplace=True)

/var/folders/c6/kf2fcpcj6pq_gwpfbfzzy81m0000gn/T/ipykernel_15182/38037239.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["FECHAPEDIDO"] = pd.to_datetime(df["FECHAPEDIDO"], dayfirst=True)


In [62]:
# assert all rows in origen follow number-number-number format
def fix_origen_typos(origen_string):
    numbers = re.findall(r"[0-9]+", origen_string)
    return "-".join(numbers)


df["ORIGEN"] = df["ORIGEN"].apply(fix_origen_typos)

In [63]:
# separate ORIGEN in three columns by '-'
origin_separated_columns = df["ORIGEN"].str.split("-", expand=True)
df["PURCHASING_REGION"] = origin_separated_columns[0]
df["PURCHASING_HOSPITAL"] = origin_separated_columns[1]
df["PURCHASING_DEPARTMENT"] = origin_separated_columns[2]
df.drop(columns=["ORIGEN"], inplace=True)

In [64]:
# remove /year from NUMERO
# df["NUMERO_PEDIDO"] = df["NUMERO"].str.split("/", expand=True)[0]
# df.drop(columns=["NUMERO"], inplace=True)

In [65]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [66]:
# basic date features
def generate_date_features(df):
    df["YEAR"] = df["FECHAPEDIDO"].dt.year
    df["MONTH"] = np.sin(2 * np.pi * df["FECHAPEDIDO"].dt.month / 12)
    df["DAYOFMONTH"] = np.sin(2 * np.pi * df["FECHAPEDIDO"].dt.day / 31)
    df["DAYOFYEAR"] = np.sin(2 * np.pi * df["FECHAPEDIDO"].dt.dayofyear / 365)
    # augmenting integer, one per day
    df = df.merge(
        (
            df[["FECHAPEDIDO"]]
            .drop_duplicates(ignore_index=True)
            .rename_axis("time_idx")
        ).reset_index(),
        on=["FECHAPEDIDO"],
    )
    return df


df = generate_date_features(df)

In [67]:
def add_timeseries_features(df):
    df["ROLLING_MEAN_3M"] = df["CANTIDADCOMPRA"].rolling(90).mean()
    df["WEIGHTED_MEAN_3M"] = (
        df["CANTIDADCOMPRA"]
        .rolling(90)
        .apply(lambda x: np.average(x, weights=range(1, len(x) + 1)))
    )
    df["ROLLING_MEAN_1Y"] = df["CANTIDADCOMPRA"].rolling(365).mean()
    df["WEIGHTED_MEAN_1Y"] = (
        df["CANTIDADCOMPRA"]
        .rolling(365)
        .apply(lambda x: np.average(x, weights=range(1, len(x) + 1)))
    )
    return df


# df = add_timeseries_features(df)

In [68]:
df.head()

,FECHAPEDIDO,NUMERO,REFERENCIA,CANTIDADCOMPRA,UNIDADESCONSUMOCONTENIDAS,PRECIO,IMPORTELINEA,TIPOCOMPRA,TGL,PRODUCTO,CODIGO_CLASS,CODIGO_NUM,PURCHASING_REGION,PURCHASING_HOSPITAL,PURCHASING_DEPARTMENT,YEAR,MONTH,DAYOFMONTH,DAYOFYEAR,time_idx
0,2015-01-02,1312/15,413568,100,5,29.150000,583.00000,Compra menor,ALMACENABLE,APOSITO DE HIDROFIBRA DE HIDROCOLOIDE C/ PLATA-3,E,64663,0,0,2,2015,0.5,0.394356,0.034422,0
1,2015-01-02,1312/15,420680,100,10,20.900000,209.00000,Compra menor,ALMACENABLE,APOSITO DE HIDROFIBRA DE HIDROCOLOIDE C/ ESPUM...,E,66071,0,0,2,2015,0.5,0.394356,0.034422,0
2,2015-01-02,1301/15,1624W,800,400,58.800000,117.60000,Compra menor,ALMACENABLE,APÓSITO TRANSPARENTE ADHESIVO-24,E,64751,0,0,2,2015,0.5,0.394356,0.034422,0
3,2015-01-02,1292/15,400403,100,10,102.803729,1028.03729,Compra menor,ALMACENABLE,SOLUCION P/ LIMPIEZA Y DESCONTAMINACION DE HER...,B,41691,0,0,2,2015,0.5,0.394356,0.034422,0
4,2015-01-05,3616/15,420680,160,10,20.900000,334.40000,Compra menor,ALMACENABLE,APOSITO DE HIDROFIBRA DE HIDROCOLOIDE C/ ESPUM...,E,66071,0,10,1,2015,0.5,0.848644,0.085965,1


In [69]:
df.to_csv(DATA_PATH + "/clean_dataset.csv", index=False)